In [23]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re



#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []
directt = []
strr = []
genres = []


#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,17500,100)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2023-04-15&runtime>0,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    #sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') != None  else np.nan
        time.append(runtime)
        
        genre = store.p.find("span",class_ = "genre").text.replace('\n', '').replace(" ","") if store.p.find("span",class_ = "genre") != None else np.nan
        genres.append(genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") !=None else np.nan
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "****"
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text if len(value) > 0 else 0
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)
        
        text=store.find("p",{"class":""}).get_text()
        text = re.sub('[(){}<>\n|]', '',text)

        if text.find("Stars:") == -1 and text.find("Star:") == -1:
            text=text + "Stars:None"
        if text.find("Director:") == -1 and text.find("Directors:") == -1:
            text = "Directors:None" + text
        text=re.split(';|\*|\n |Stars:|Star:|Director:|Directors:',text)    

        directors = text[1].strip(" ")
        stars = text[2]
        
        directt.append(directors)
        strr.append(stars)
        
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)

#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross": gross, 
                           "Description": description, "Directors":directt,"Stars":strr,"Genres":genres  })
movie_list.head(5)

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description,Directors,Stars,Genres
0,John Wick: Chapter 4,(2023),169 min,8.0,78,"197,340",%^%^%^,John Wick uncovers a path to defeating The Hig...,Chad Stahelski,"Keanu Reeves, Laurence Fishburne, George Georg...","Action,Crime,Thriller"
1,Dungeons & Dragons: Honor Among Thieves,(2023),134 min,7.3,72,"124,542",%^%^%^,A charming thief and a band of unlikely advent...,"John Francis Daley, Jonathan Goldstein","Chris Pine, Michelle Rodriguez, Regé-Jean Page...","Action,Adventure,Comedy"
2,The Super Mario Bros. Movie,(2023),92 min,7.2,46,"136,183",%^%^%^,A plumber named Mario travels through an under...,"Aaron Horvath, Michael Jelenic, Pierre Leduc, ...","Chris Pratt, Anya Taylor-Joy, Charlie Day, Jac...","Animation,Adventure,Comedy"
3,Air,(I) (2023),111 min,7.5,73,"97,982",%^%^%^,Follows the history of shoe salesman Sonny Vac...,Ben Affleck,"Matt Damon, Jason Bateman, Ben Affleck, Chris ...","Drama,Sport"
4,Hypnotic,(2023),93 min,5.4,53,"9,311",%^%^%^,A detective investigates a mystery involving h...,Robert Rodriguez,"Ben Affleck, Alice Braga, JD Pardo, Dayo Okeniyi","Action,Mystery,Thriller"


233

In [366]:
movie_list.to_excel("IMBD_data_2022-2023.xlsx")

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description,Directors,Stars
17495,Unidentified Objects,(2022),100 min,7.2,****,87,%^%^%^,"An uptight dwarf and his free-spirited, alien-...",Juan Felipe Zuleta,"Sarah Hay, Matthew Jeffers, Roy Abramsohn, Ham..."
17496,Deep Water,(I) (2022),115 min,5.4,53,"50,253",%^%^%^,A well-to-do husband who allows his wife to ha...,Adrian Lyne,"Ben Affleck, Ana de Armas, Tracy Letts, Grace ..."
17497,Selfiee,(2023),148 min,5.9,****,"11,664",%^%^%^,Bollywood superstar Vijay Kumar needs to obtai...,Raj Mehta,"Akshay Kumar, Mrunal Thakur, Nushrratt Bharucc..."
17498,Demon Slayer: Kimetsu No Yaiba - To the Swords...,(2023),110 min,6.7,****,"4,670",%^%^%^,All the Upper Rank Demons assemble at the Infi...,Haruo Sotozaki,"Zach Aguilar, Laila Berzins, Kira Buckland, Re..."
17499,How to Blow Up a Pipeline,(2022),104 min,7.3,77,"2,160",%^%^%^,A crew of environmental activists plot a darin...,Daniel Goldhaber,"Ariela Barer, Kristine Froseth, Lukas Gage, Fo..."


In [377]:
movie_list["Year of Release"].value_counts()

(2022)           10570
(2023)            5350
(I) (2022)         833
(II) (2022)        333
(II) (2023)        176
(V) (2022)          89
(III) (2022)        56
(I) (2023)          40
(IV) (2022)         22
(III) (2023)         8
                     8
(VI) (2022)          4
(VII) (2022)         4
(IX) (2022)          2
(IV) (2023)          2
(VIII) (2022)        1
(X) (2022)           1
(V) (2023)           1
Name: Year of Release, dtype: int64

In [25]:
pure_year = []
for a in movie_list["Year of Release"]:
    a=re.sub('[(){}<>\n|(I)(II)(III)(X)(V) ]', '',a)
    pure_year.append(a)
len(pure_year)   

17500

In [26]:
movie_list["Year_ONLY"] = pure_year
movie_list["Year_ONLY"].value_counts()

2022    12220
2023     5273
            7
Name: Year_ONLY, dtype: int64

In [396]:
#movie_list.to_excel("IMDB_2020-2023.xlsx")  #Getting data as excel

In [50]:
dff=movie_list.copy()


In [55]:
missing_genres=dff[dff.Genres.isna()].index
all_genres = []
for a,k in dff.iterrows():
    if a not in missing_genres:
        all_genres.extend(k["Genres"].split(","))
    
all_genres=list(set(all_genres))

for genre in all_genres:
    for a,k in dff.iterrows():
        dff.loc[a,genre] = 1.0 if a not in missing_genres and genre in dff.loc[a,"Genres"] else 0.0

    
    



In [68]:
dff.iloc[:,12:].sum()

Romance       1387.0
Music          294.0
Musical         96.0
Adventure     1994.0
Horror        3134.0
Action        3605.0
Sport          327.0
Sci-Fi         726.0
Reality-TV       5.0
Comedy        4806.0
History        718.0
War            239.0
Family         496.0
Mystery       1580.0
Crime         1540.0
Thriller      3845.0
News             1.0
Biography      482.0
Drama         8376.0
Fantasy        797.0
Western        132.0
Animation      558.0
dtype: float64

In [69]:
dff.columns.shape

(34,)

In [397]:
movie_list.columns

Index(['Movie Name', 'Year of Release', 'Watch Time', 'Movie Rating',
       'Meatscore of movie', 'Votes', 'Gross', 'Description', 'Directors',
       'Stars', 'Year_ONLY'],
      dtype='object')

In [71]:
dff.to_excel("IMDB-2022-2023-W-Genres.xlsx")

In [398]:
movie_list["Movie Name"].value_counts() #bazı filmlerin isimlerinden çok var

Babylon                     77
Aftersun                    77
Smile                       77
Fall                        77
From Black                  76
                            ..
Malvada                      1
The Burned Over District     1
Unborn                       1
Disfluency                   1
La mia danza                 1
Name: Movie Name, Length: 9893, dtype: int64

In [401]:
movie_list[movie_list["Movie Name"] =="Babylon"].loc[5641].Stars bu şekilde datanın doğrluğu test edilebilir

'Dana Kjeldsen, Alex Niforatos, Tyler Hunt, Shanah Cooley'

In [402]:
movie_list["Watch Time"].value_counts()

90 min       508
102 min      494
100 min      454
104 min      438
105 min      422
            ... 
188 min        1
5,538 min      1
14 min         1
39 min         1
257 min        1
Name: Watch Time, Length: 184, dtype: int64

In [405]:
movie_list["Movie Rating"].value_counts()

7.2    710
6.1    666
6.7    658
7.0    620
7.3    619
      ... 
1.6      3
1.9      3
1.3      2
1.2      1
1.1      1
Name: Movie Rating, Length: 91, dtype: int64

In [406]:
movie_list.head()

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description,Directors,Stars,Year_ONLY
0,Evil Dead Rise,(2023),96 min,7.1,69,"34,865",%^%^%^,A twisted tale of two estranged sisters whose ...,Lee Cronin,"Mirabai Pease, Richard Crouchley, Anna-Maree T...",2023
1,The Super Mario Bros. Movie,(2023),92 min,7.3,46,"91,813",%^%^%^,The story of The Super Mario Bros. on their jo...,"Aaron Horvath, Michael Jelenic, Pierre Leduc, ...","Chris Pratt, Anya Taylor-Joy, Charlie Day, Jac...",2023
2,Beau Is Afraid,(2023),179 min,7.2,62,"11,792",%^%^%^,"Following the sudden death of his mother, a mi...",Ari Aster,"Joaquin Phoenix, Patti LuPone, Amy Ryan, Natha...",2023
3,Scream VI,(2023),122 min,6.7,61,"64,731",%^%^%^,"In the next installment, the survivors of the ...","Matt Bettinelli-Olpin, Tyler Gillett","Courteney Cox, Melissa Barrera, Jenna Ortega, ...",2023
4,John Wick: Chapter 4,(2023),169 min,8.2,78,"142,011",%^%^%^,John Wick uncovers a path to defeating The Hig...,Chad Stahelski,"Keanu Reeves, Laurence Fishburne, George Georg...",2023


In [1]:
import pandas as pd

In [20]:
dfff = pd.read_excel("IMDB_2020-2023.xlsx",index_col=0)
dfff

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Gross,Description,Directors,Stars,Year_ONLY
0,Evil Dead Rise,(2023),96 min,7.1,69,"34,865",%^%^%^,A twisted tale of two estranged sisters whose ...,Lee Cronin,"Mirabai Pease, Richard Crouchley, Anna-Maree T...",2023.0
1,The Super Mario Bros. Movie,(2023),92 min,7.3,46,"91,813",%^%^%^,The story of The Super Mario Bros. on their jo...,"Aaron Horvath, Michael Jelenic, Pierre Leduc, ...","Chris Pratt, Anya Taylor-Joy, Charlie Day, Jac...",2023.0
2,Beau Is Afraid,(2023),179 min,7.2,62,"11,792",%^%^%^,"Following the sudden death of his mother, a mi...",Ari Aster,"Joaquin Phoenix, Patti LuPone, Amy Ryan, Natha...",2023.0
3,Scream VI,(2023),122 min,6.7,61,"64,731",%^%^%^,"In the next installment, the survivors of the ...","Matt Bettinelli-Olpin, Tyler Gillett","Courteney Cox, Melissa Barrera, Jenna Ortega, ...",2023.0
4,John Wick: Chapter 4,(2023),169 min,8.2,78,"142,011",%^%^%^,John Wick uncovers a path to defeating The Hig...,Chad Stahelski,"Keanu Reeves, Laurence Fishburne, George Georg...",2023.0
...,...,...,...,...,...,...,...,...,...,...,...
17495,Unidentified Objects,(2022),100 min,7.2,****,87,%^%^%^,"An uptight dwarf and his free-spirited, alien-...",Juan Felipe Zuleta,"Sarah Hay, Matthew Jeffers, Roy Abramsohn, Ham...",2022.0
17496,Deep Water,(I) (2022),115 min,5.4,53,"50,253",%^%^%^,A well-to-do husband who allows his wife to ha...,Adrian Lyne,"Ben Affleck, Ana de Armas, Tracy Letts, Grace ...",2022.0
17497,Selfiee,(2023),148 min,5.9,****,"11,664",%^%^%^,Bollywood superstar Vijay Kumar needs to obtai...,Raj Mehta,"Akshay Kumar, Mrunal Thakur, Nushrratt Bharucc...",2023.0
17498,Demon Slayer: Kimetsu No Yaiba - To the Swords...,(2023),110 min,6.7,****,"4,670",%^%^%^,All the Upper Rank Demons assemble at the Infi...,Haruo Sotozaki,"Zach Aguilar, Laila Berzins, Kira Buckland, Re...",2023.0


In [16]:
dfff.columns

Index(['Unnamed: 0', 'Movie Name', 'Year of Release', 'Watch Time',
       'Movie Rating', 'Meatscore of movie', 'Votes', 'Gross', 'Description',
       'Directors', 'Stars', 'Year_ONLY'],
      dtype='object')

In [22]:
dfff.dtypes

Movie Name             object
Year of Release        object
Watch Time             object
Movie Rating          float64
Meatscore of movie     object
Votes                  object
Gross                  object
Description            object
Directors              object
Stars                  object
Year_ONLY             float64
dtype: object

In [411]:
dfff.Gross.value_counts()

%^%^%^      16028
$659.68M       76
$168.15M       76
$45.63M        76
$151.04M       76
$376.85M       76
$343.26M       76
$453.72M       76
$90.23M        76
$68.28M        76
$93.88M        76
$369.35M       76
$168.46M       76
$103.37M       76
$62.47M        76
$81.64M        76
$123.28M       76
$718.73M       76
$72.86M        76
$14.50M        76
$0.01M          3
$0.00M          2
$411.33M        1
$67.33M         1
$0.08M          1
$0.19M          1
$0.06M          1
$0.02M          1
$0.12M          1
$12.78M         1
$6.60M          1
$0.81M          1
$118.31M        1
$9.59M          1
$369.70M        1
$73.87M         1
$46.89M         1
$22.78M         1
$93.66M         1
$97.23M         1
$31.85M         1
$95.85M         1
$190.87M        1
$105.34M        1
$148.65M        1
Name: Gross, dtype: int64